In [18]:
## Basic imports
import sys
import os
import time
import pixiedust

# WARNING: These global declarations cause the parallel implementation to 
# crash when executed on Windows
connections = None
networks = None
flowdepthvel = None

from sys import platform
if platform == "linux" or platform == "linux2":
    pass
elif platform == "darwin":
    pass
elif platform == "win32":
    print('The parallel version of compute_nhd_routing.py will not execute as currently')
    print('written due to the lack of a fork() capability in the windows OS.')
    print('For parallel execution, please us a *nix OS.')
    print('\nexiting...')
    sys.exit()
    # Some useful references:
    # https://stackoverflow.com/questions/985281/what-is-the-closest-thing-windows-has-to-fork/985525#985525
    # https://stackoverflow.com/questions/8220108/how-do-i-check-the-operating-system-in-python
    # https://stackoverflow.com/questions/6596617/python-multiprocess-diff-between-windows-and-linux

ENV_IS_CL = False
if ENV_IS_CL: root = '/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/'
elif not ENV_IS_CL: 
    sys.setrecursionlimit(4000)
    #root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
    root=os.path.abspath('')
    #print(root)
    root=os.path.dirname(os.path.abspath(''))
    #print(root)    
    root=os.path.dirname(os.path.dirname(os.path.abspath('')))
    #print(root)    
    #root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
    print(root)
    
    sys.path.append(os.path.join(root, r'src', r'python_framework'))
    fortran_source_dir = os.path.join(root, r'src', r'fortran_routing', r'mc_pylink_v00', r'MC_singleRCH_singleTS')
#     print(fortran_source_dir) 
    sys.path.append(fortran_source_dir)
print(sys.path)
from mc_singleCh_SingleTStep import compute_mc_reach_up2down 
# import mc_sc_stime as mc
# import mc_sseg_stime_noIC as mc

###############################################
# Build supernetwork_data, supernetwork_values
###############################################

import nhd_network_utilities as nnu
import nhd_reach_utilities as nru

## Muskingum Cunge
import numpy as np

if 1==1:
#     global connections
#     global networks
#     global flowdepthvel

    verbose = True
    debuglevel = 0
    showtiming = True

    test_folder = os.path.join(root, r'test')
    geo_input_folder = os.path.join(test_folder, r'input', r'geo', r'Channels')
    print(geo_input_folder)

    #TODO: Make these commandline args
    """##NHD Subset (Brazos/Lower Colorado)"""
#    supernetwork = 'Brazos_LowerColorado_ge5'  #this
    supernetwork= 'Pocono_TEST1'
    """##NHD CONUS order 5 and greater"""
    # supernetwork = 'CONUS_ge5'
    """These are large -- be careful"""
#     supernetwork = 'Mainstems_CONUS'
    # supernetwork = 'CONUS_FULL_RES_v20'
    # supernetwork = 'CONUS_Named_Streams' #create a subset of the full resolution by reading the GNIS field
    # supernetwork = 'CONUS_Named_combined' #process the Named streams through the Full-Res paths to join the many hanging reaches

    if verbose: print('creating supernetwork connections set')
    if showtiming: start_time = time.time()
    #STEP 1
    supernetwork_data, supernetwork_values = nnu.set_networks(
        supernetwork = supernetwork
        , geo_input_folder = geo_input_folder
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        )
    if verbose: print('supernetwork connections set complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))
        

#############################        
# Build networks
#############################            

if 1==1: 
    #STEP 2
    if showtiming: start_time = time.time()
    if verbose: print('organizing connections into networks and reaches ...')
    networks = nru.compose_reaches(
        supernetwork_values
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        , showtiming = showtiming
        )
    if verbose: print('reach organization complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))
        
        
######################
# Build connections
######################

if 1==1:
#STEP 3
    if showtiming: start_time = time.time()
    executiontype = 'serial' # 'parallel'

    if verbose: print('executing serial computation on ordered reaches ...')
    connections = supernetwork_values[0]
it=0
############################
# Build ordered_reaches
############################
# Actually, all keys of networks dictionary are terminal_segment, the value of 'terminal_segment:' is 
# the key value itself. Now, network takes networks values corresponding to 'terminal_segment' keys. 
for terminal_segment, network in networks.items():

#     print(f"for-loop: it-> {it} terminal_segment-> {terminal_segment}")
#     print(f"for-loop: network-> {network}")
#     print(f"for-loop: network-> {network['maximum_reach_seqorder']}")
#     print()
#     if verbose: print(f"Executing simulation on network {terminal_segment} beginning with streams of order {network['maximum_reach_seqorder']}")
    ordered_reaches = {}
    reach_flowdepthvel = {}
    last_segment_reach={}
    
    # head_segment = links that are either junctions or headwater_reaches belonged to terminal_segment link
    for head_segment, reach in network['reaches'].items():
#         print(f"    for-loop: head_segment-> {head_segment} ,----, reaches-> {reach}")
#         print()
#         print(f"    reach['seqorder']-> {reach['seqorder']},----, ordered_reaches-> {ordered_reaches}")
#         print()
        if reach['seqorder'] not in ordered_reaches:
            ordered_reaches.update({reach['seqorder']:[]}) #TODO: Should this be a set/dictionary?
#             print(f"        ordered_reaches-> {ordered_reaches}")
#             print()            
        ordered_reaches[reach['seqorder']].append([head_segment, reach])
#         print(f"    ordered_reaches-> {ordered_reaches}")
#         print()  

# DONGHA NEW LINE        
        # store last segment link ID for each reach referenced by head_segment
        seg_list=list(reach['segments_list'])
#         segnb=len(reach['segments'])
        lastseg=seg_list[0]
        last_segment_reach[head_segment]= lastseg
#         print(f"last_segment_reach-> {last_segment_reach}")
#         print(f"--------------------------------")
    ## test
    it=it+1
    if it==197:
        break
print(network)
print()
print(ordered_reaches)

/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing
['/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/python_routing_v02', '/home/dongha.kim/anaconda3/lib/python37.zip', '/home/dongha.kim/anaconda3/lib/python3.7', '/home/dongha.kim/anaconda3/lib/python3.7/lib-dynload', '', '/home/dongha.kim/.local/lib/python3.7/site-packages', '/home/dongha.kim/anaconda3/lib/python3.7/site-packages', '/home/dongha.kim/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/dongha.kim/.ipython', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/python_framework', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/fortran_routing/mc_pylink_v00/MC_singleRCH_singleTS', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/python_framework', '/home/dongha.kim/github/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src/fortr

# Version 1: Time loop is the top loop

In [19]:
# %%pixie_debugger 
import mc_sseg_stime_noIC as mc
from time import process_time 

if 1==1:
    print(network['maximum_reach_seqorder'])
    print()
    t1_start = process_time()  
    
    nts=1000
    dt_mc= 300.0  #currently used MC simulation time interval 
    ## START: improvised lateral flow
    bply=nts/2.0; aply= -0.2
    ## END    

    flowdepthvel = {connection:{'flow':np.zeros(nts)
                                , 'depth':np.zeros(nts)
                                , 'vel':np.zeros(nts)
                                , 'qlat':np.zeros(nts)}
                               for connection in connections}         
    
    for ts in range (0,nts):
        #print(f'timestep: {ts}\n')
        #Note that not networks but network that takes networks values corresponding to 'terminal_segment' keys.
        #Also, network here takes 'terminal_segment' key only at the lower end of a network.      
        for x in range(network['maximum_reach_seqorder'],-1,-1):    
            for head_segment, reach in ordered_reaches[x]:               
                ## Store last segment numbers for upstream reaches for a current head_segment reach 
                usrch_list=list(reach['upstream_reaches'])
                if reach['upstream_reaches']=={0}:
                    usrchnb=0
                else:
                    usrchnb= len(reach['upstream_reaches'])
                    
                ## channel data inputs
                seg_list=list(reach['segments_list'])
                seg_list=seg_list[::-1] #to reversed order
                ncomp= len(reach['segments_list']); mc.var.ncomp=ncomp
    #UNDER these for-loops
    #for ts in range (0,nts):
        #for x in range(network['maximum_reach_seqorder'],-1,-1):
            #for head_segment, reach in ordered_reaches[x]:                
                for seg in range(0,ncomp):   
                    
                    segID= seg_list[seg]       
                    mc.var.dt=  dt_mc 
                    mc.var.dx= connections[segID]['data'][supernetwork_data['length_col']]
                    mc.var.bw= connections[segID]['data'][supernetwork_data['bottomwidth_col']] 
                    mc.var.tw=  connections[segID]['data'][supernetwork_data['topwidth_col']]
                    mc.var.twcc= connections[segID]['data'][supernetwork_data['topwidthCC_col']]
                    mc.var.n= connections[segID]['data'][supernetwork_data['manningn_col']]
                    mc.var.ncc= connections[segID]['data'][supernetwork_data['manningnCC_col']]
                    mc.var.cs= connections[segID]['data'][supernetwork_data['ChSlp_col']]
                    mc.var.so= connections[segID]['data'][supernetwork_data['slope_col']]
                    mc.var.ql= aply*(ts+1 - bply)**2.0 - aply*bply**2.0 + (seg+1)*10.0
                    # initial values of MC parameters when time is zero and beyond
                    if ts==0:
                        mc.var.qup=0.0
                        mc.var.quc=0.0
                        mc.var.qdp=0.0
                        mc.var.qdc=0.0
                        mc.var.vel=0.0
                        mc.var.depth=-999.0
                    else:
                        if seg>0:
                            mc.var.qup= flowdepthvel[seg_list[seg-1]]['flow'][ts-1]
                            mc.var.quc= mc.var.qup
                            #mc.var.qdp= flowdepthvel[seg_list[seg]]['flow'][ts-1]
                        elif reach['upstream_reaches'] != {0}:
                            ## when it is first segment of head_segment's reach, compute previous flow from
                            ## the last segments of upstream reaches by adding the flows if the reaches are 
                            ## more than 1.
                            mc.var.qus_prev=0.0
                            for usrch in range(0,usrchnb):
                                uslsegID= last_segment_reach[usrch_list[usrch]]              
                                mc.var.qus_prev= mc.var.qus_prev + flowdepthvel[uslsegID]['flow'][ts-1]
                            
                            mc.var.qup= mc.var.qus_prev
                            mc.var.quc= mc.var.qup
                            #mc.var.qdp= flowdepthvel[seg_list[seg]]['flow'][ts-1]
                        elif reach['upstream_reaches'] == {0}:
                            mc.var.qup= 0.0
                            mc.var.quc= 0.0
                            #mc.var.qdp= flowdepthvel[seg_list[seg]]['flow'][ts-1]                  
                        mc.var.qdp= flowdepthvel[segID]['flow'][ts-1]     
                        mc.var.qdc= 0.0
                        mc.var.vel= flowdepthvel[segID]['vel'][ts-1]  
                        mc.var.depth= flowdepthvel[segID]['depth'][ts-1]     
#                     print(f"ts {ts} head_segment {head_segment} segINDEX {seg} segID {segID} ql {mc.var.ql} INI:qup quc qdp qdc vel depth {mc.var.qup} {mc.var.quc} {mc.var.qdp} {mc.var.qdc} {mc.var.vel} {mc.var.depth}")
                    
                    # call Fortran routines
                    mc.mc.main()

#                     print(f"ts {ts} head_segment {head_segment} segINDEX {seg} segID {segID} ql {mc.var.ql} FNL:qup quc qdp qdc vel depth {mc.var.qup} {mc.var.quc} {mc.var.qdp} {mc.var.qdc} {mc.var.vel} {mc.var.depth}")
  
                    # store computed values at time k for computation at k+1
                    #qd[k,i,j]: k=0/1: previous/current timestep; i: node ID; j=0/1: upstream/current reach
                    #mc.var.qd[0,i,1]= mc.var.qd[1,i,1]
                    #output keeping
                    flowdepthvel[segID]['flow'][ts]= mc.var.qdc # mc.var.qd[1,seg] #wqd[k,i]= mc.var.qd[1,seg]
                    flowdepthvel[segID]['vel'][ts]= mc.var.vel # mc.var.vela[1,seg]  #wvela[k,i]= mc.var.vela[1,seg]
                    flowdepthvel[segID]['depth'][ts]= mc.var.depth # mc.var.deptha[1,seg] #wdeptha[k,i]= mc.var.deptha[1,seg] 

    # Stop the stopwatch / counter 
    t1_stop = process_time() 
   
    print("Elapsed time:", t1_stop, t1_start)     
    print("Elapsed time during the whole program in seconds:", t1_stop-t1_start) 

4

Elapsed time: 123.535005731 123.134266209
Elapsed time during the whole program in seconds: 0.40073952199999496


In [15]:
print(ordered_reaches)

{0: [[1562342, {'reach_tail': 1563610, 'downstream_reach': 0, 'reach_head': 1562342, 'seqorder': 0, 'segments': {1563136, 1562626, 1562628, 1563144, 1562632, 1563146, 1562636, 1562640, 1562642, 1562650, 1562652, 1563170, 1562660, 1562662, 1562666, 1562668, 1562670, 1563188, 1562676, 1563190, 1562680, 1563194, 1562682, 1562684, 1562688, 1562692, 1562700, 1562702, 1562704, 1562708, 1562712, 1562714, 1562718, 1562730, 1562734, 1562736, 1562754, 1562758, 1562762, 1562764, 1563132, 1565902, 1565904, 1565906, 1562340, 1562342, 1562860, 1562866, 1562354, 1562870, 1562872, 1562876, 1562880, 1562392, 1562928, 1562950, 1562456, 1562498, 1563548, 1563052, 1563064, 1563066, 1563592, 1563596, 1563600, 1563602, 1563604, 1563092, 1563606, 1563610, 1563098, 1563614, 1563616, 1562592, 1563618, 1563622, 1563110, 1563624, 1563626, 1563114, 1563116, 1563628, 1562606, 1562610, 1562612, 1562616, 1562620}, 'segments_list': [1563610, 1563604, 1563602, 1563600, 1563596, 1563592, 1563606, 1563614, 1563618, 1563

# version 2: Time loop under segment loop

In [22]:
# %%pixie_debugger 
#########################################
# time loop squeezed under segment loop
#########################################
import mc_sseg_stime_noIC as mc
from time import process_time 

if 1==1:
    print(network['maximum_reach_seqorder'])
    print()
    t1_start = process_time()  

   
    dt_mc= 300.0  #currently used MC simulation time interval 
    nts=1000
    ## START: improvised lateral flow
    bply=nts/2.0; aply= -0.2
    ## END    

    flowdepthvel = {connection:{'flow':np.zeros(nts)
                                , 'depth':np.zeros(nts)
                                , 'vel':np.zeros(nts)
                                , 'qlat':np.zeros(nts)}
                               for connection in connections}         
    
#     for ts in range (0,nts):
        #print(f'timestep: {ts}\n')
        #Note that not networks but network that takes networks values corresponding to 'terminal_segment' keys.
        #Also, network here takes 'terminal_segment' key only at the lower end of a network.      
    for x in range(network['maximum_reach_seqorder'],-1,-1):    
        for head_segment, reach in ordered_reaches[x]:               
            ## Store last segment numbers for upstream reaches for a current head_segment reach 
            usrch_list=list(reach['upstream_reaches'])
            if reach['upstream_reaches']=={0}:
                usrchnb=0
            else:
                usrchnb= len(reach['upstream_reaches'])
                    
            ## channel data inputs
            seg_list=list(reach['segments_list'])
            seg_list=seg_list[::-1] #to reversed order
            ncomp= len(reach['segments_list']); mc.var.ncomp=ncomp
               
            for seg in range(0,ncomp):                   
                segID= seg_list[seg]       
                mc.var.dt=  dt_mc 
                mc.var.dx= connections[segID]['data'][supernetwork_data['length_col']]
                mc.var.bw= connections[segID]['data'][supernetwork_data['bottomwidth_col']] 
                mc.var.tw=  connections[segID]['data'][supernetwork_data['topwidth_col']]
                mc.var.twcc= connections[segID]['data'][supernetwork_data['topwidthCC_col']]
                mc.var.n= connections[segID]['data'][supernetwork_data['manningn_col']]
                mc.var.ncc= connections[segID]['data'][supernetwork_data['manningnCC_col']]
                mc.var.cs= connections[segID]['data'][supernetwork_data['ChSlp_col']]
                mc.var.so= connections[segID]['data'][supernetwork_data['slope_col']]
                
                for ts in range (0,nts):             
                    mc.var.ql= aply*(ts+1 - bply)**2.0 - aply*bply**2.0 + (seg+1)*10.0
                    # initial values of MC parameters when time is zero and beyond
                    if ts==0:
                        mc.var.qup=0.0
                        mc.var.quc=0.0
                        mc.var.qdp=0.0
                        mc.var.qdc=0.0
                        mc.var.vel=0.0
                        mc.var.depth=-999.0
                    else:
                        if seg>0:
                            mc.var.qup= flowdepthvel[seg_list[seg-1]]['flow'][ts-1]
                            mc.var.quc= mc.var.qup
                            #mc.var.qdp= flowdepthvel[seg_list[seg]]['flow'][ts-1]
                        elif reach['upstream_reaches'] != {0}:
                            ## when it is first segment of head_segment's reach, compute previous flow from
                            ## the last segments of upstream reaches by adding the flows if the reaches are 
                            ## more than 1.
                            mc.var.qus_prev=0.0
                            for usrch in range(0,usrchnb):
                                uslsegID= last_segment_reach[usrch_list[usrch]]              
                                mc.var.qus_prev= mc.var.qus_prev + flowdepthvel[uslsegID]['flow'][ts-1]
                            
                            mc.var.qup= mc.var.qus_prev
                            mc.var.quc= mc.var.qup
                            #mc.var.qdp= flowdepthvel[seg_list[seg]]['flow'][ts-1]
                        elif reach['upstream_reaches'] == {0}:
                            mc.var.qup= 0.0
                            mc.var.quc= 0.0
                            #mc.var.qdp= flowdepthvel[seg_list[seg]]['flow'][ts-1]                  
                        mc.var.qdp= flowdepthvel[segID]['flow'][ts-1]     
                        mc.var.qdc= 0.0
                        mc.var.vel= flowdepthvel[segID]['vel'][ts-1]  
                        mc.var.depth= flowdepthvel[segID]['depth'][ts-1]     
#                     print(f"ts {ts} head_segment {head_segment} segINDEX {seg} segID {segID} ql {mc.var.ql} INI:qup quc qdp qdc vel depth {mc.var.qup} {mc.var.quc} {mc.var.qdp} {mc.var.qdc} {mc.var.vel} {mc.var.depth}")
                    
                    # call Fortran routines
                    mc.mc.main()

#                     print(f"ts {ts} head_segment {head_segment} segINDEX {seg} segID {segID} ql {mc.var.ql} FNL:qup quc qdp qdc vel depth {mc.var.qup} {mc.var.quc} {mc.var.qdp} {mc.var.qdc} {mc.var.vel} {mc.var.depth}")
  
                    # store computed values at time k for computation at k+1
                    #output keeping
                    flowdepthvel[segID]['flow'][ts]= mc.var.qdc # mc.var.qd[1,seg] #wqd[k,i]= mc.var.qd[1,seg]
                    flowdepthvel[segID]['vel'][ts]= mc.var.vel # mc.var.vela[1,seg]  #wvela[k,i]= mc.var.vela[1,seg]
                    flowdepthvel[segID]['depth'][ts]= mc.var.depth # mc.var.deptha[1,seg] #wdeptha[k,i]= mc.var.deptha[1,seg] 

    # Stop the stopwatch / counter 
    t1_stop = process_time() 
   
    print("Elapsed time:", t1_stop, t1_start)     
    print("Elapsed time during the whole program in seconds:", t1_stop-t1_start) 

4

Elapsed time: 127.879312617 127.628739636
Elapsed time during the whole program in seconds: 0.2505729809999906
